In [1]:
library(DESeq2)
library(EnhancedVolcano)
library(clusterProfiler)
library(enrichplot)
library(ggplot2)
organism = "org.Mm.eg.db"
library(organism, character.only = TRUE)
library(DOSE)


Warning message:
“package ‘DESeq2’ was built under R version 4.3.3”
Loading required package: S4Vectors

Warning message:
“package ‘S4Vectors’ was built under R version 4.3.3”
Loading required package: stats4

Loading required package: BiocGenerics

Warning message:
“package ‘BiocGenerics’ was built under R version 4.3.2”

Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    find

In [2]:
df <- read.table("dea/sum.tsv", header=TRUE, sep="\t")
# Drop duplicates in id column
df <- df[!duplicated(df$id),]
rownames(df) <- df$id
df$id <- NULL

annotation <- read.table("dea/annotation.bed", header=FALSE, sep="\t", col.names = c("chr", "start", "end", "name", "score", "strand", "type", "gene.id", "gene", "transcripts", "databases"))
# Remove chr, start, end, score, strand, gene.id, transcripts
annotation$chr <- NULL
annotation$start <- NULL
annotation$end <- NULL
annotation$score <- NULL
annotation$strand <- NULL
annotation$gene.id <- NULL
annotation$transcripts <- NULL

rownames(annotation) <- annotation$name
annotation$name <- NULL

phenotype <- read.csv("dea/phenotype.csv", header=TRUE, row.names = 1)
phenotype$transgene <- as.factor(phenotype$transgene)
phenotype$drug <- as.factor(phenotype$drug)

# Center and scale age and induction
phenotype$age <- scale(phenotype$age)
phenotype$induction <- scale(phenotype$induction)

df_genes <- read.table("dea/gene_tpm.tsv", sep="\t", header=TRUE, row.names=1)
df_genes$gene_name <- NULL
# Order genes columns like phenotype rows
df_genes <- df_genes[,rownames(phenotype)]

phenotype$esr1 <- scale(as.numeric(df_genes["Esr1",]))

In [3]:
run_analysis <- function(directory, phenotype = NULL, design = NULL, dds = NULL, contrast = NULL, name = NULL) {
    # Fail if both contrast and name are NULL
    if (is.null(contrast) && is.null(name)) {
        stop("Both contrast and name cannot be NULL")
    }
    # Fail if none of contrast and name are NULL
    if (!is.null(contrast) && !is.null(name)) {
        stop("Both contrast and name cannot be provided")
    }

    if (is.null(phenotype) && is.null(design) && is.null(dds)) {
        stop("Either phenotype and design or dds must be provided")
    }

    if (is.null(phenotype) != is.null(design)) {
        stop("Both phenotype and design must be provided")
    }

    if (!is.null(dds) && !is.null(phenotype)) {
        stop("Either dds or phenotype and design must be provided")
    }

    alpha <- 0.05

    if (is.null(dds)) {
        dds <- DESeqDataSetFromMatrix(countData = df[, rownames(phenotype)],
                                    colData = phenotype,
                                    design = design)
        dds <- DESeq(dds)
    }

    if (!is.null(contrast)) {
        res <- results(dds, contrast = contrast, alpha = alpha, lfcThreshold = 2, altHypothesis = "greaterAbs")
    } else {
        res <- results(dds, name = name, alpha = alpha, lfcThreshold = 2, altHypothesis = "greaterAbs")
    }
    res <- res[order(res$padj),]
    res <- cbind(res, annotation[rownames(res),])

    # Recreate directory if it exists
    if (dir.exists(directory)) {
        unlink(directory, recursive = TRUE)
    }
    dir.create(directory, showWarnings = FALSE)

    write.table(res, file=paste0(directory, "/res.tsv"), sep="\t", col.names=NA, row.names=TRUE, quote=FALSE)

    EnhancedVolcano(res,
        x='log2FoldChange',
        y='padj',
        lab=res$gene,
        pCutoff = alpha)
    ggsave(paste0(directory, "/volcano.png"))

    # Keep only rows without any NA
    res <- res[complete.cases(res),]

    sign <- res[res$padj < alpha,]

    genes <- unique(unlist(strsplit(sign$gene, ",")))
    write.table(genes, file=paste0(directory, "/genes.txt"), row.names=FALSE, col.names=FALSE, quote=FALSE)
    db <- unique(unlist(strsplit(sign$databases, ",")))
    write.table(db, file=paste0(directory, "/db.txt"), row.names=FALSE, col.names=FALSE, quote=FALSE)

    gene_list <- sign$log2FoldChange
    names(gene_list) <- sign$gene
    gene_list = sort(gene_list, decreasing = TRUE)

    gse <- gseGO(geneList=gene_list,
            ont ="ALL", 
            keyType = "SYMBOL",
            pvalueCutoff = 0.05,
            verbose = TRUE,
            OrgDb = organism,
            pAdjustMethod = "BH")

    # If no significant GO terms, return
    if (nrow(as.data.frame(gse)) > 0) {
        dotplot(gse, showCategory=10, split=".sign") + facet_grid(.~.sign)
        ggsave(paste0(directory, "/dot.png"))

        gse <- pairwise_termsim(gse)
        emapplot(gse, showCategory = 10)
        ggsave(paste0(directory, "/emap.png"))

        cnetplot(gse, categorySize="pvalue", foldChange=gene_list, showCategory = 3)
        ggsave(paste0(directory, "/cnet.png"))

        ridgeplot(gse) + labs(x = "enrichment distribution")
        ggsave(paste0(directory, "/ridge.png"))
    } else {
        print("No significant GO terms")
    }

    return(dds)
}

# Aging

In [4]:
dds_esr1 <- run_analysis(
    "dea/age",
    phenotype = phenotype,
    design = ~ age + transgene + induction + drug + esr1,
    name = "age"
)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

5 rows did not converge in beta, labelled in mcols(object)$betaConv. Use larger maxit argument with nbinomWaldTest

Saving 6.67 x 6.67 in image
preparing geneSet collections...

GSEA analysis...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (41.78% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are duplicate gene names, fgsea may produce unexpected results.”
no term enriched under specific pvalueCutoff...



[1] "No significant GO terms"


In [5]:
cor.test(as.numeric(df[1,]), phenotype$esr1)$p.value

[1] 0.01361888

# ESR1 association

In [6]:
correlations <- t(apply(df, 1, function(row) {
    res <- cor.test(as.numeric(row), phenotype$esr1)
    return(c(res$estimate, res$p.value))
}))
correlations <- as.data.frame(correlations)
colnames(correlations) <- c("correlation", "p.value")
correlations$p.adj <- p.adjust(correlations$p.value, method = "BH")
correlations <- correlations[order(correlations$correlation),]
correlations

,correlation,p.value,p.adj
,<dbl>,<dbl>,<dbl>
chr14:87047771-87072537:-,-0.3978165,0.0005391544,0.2632107
chr15:44259906-44269388:-,-0.3899490,0.0007091335,0.2632107
chr3:55588318-55631070:-,-0.3779405,0.0010637002,0.2632107
chr7:19266598-19266682:+,-0.3616702,0.0017989426,0.2632107
chr7:101794764-101825612:-,-0.3536681,0.0023067755,0.2632107
chr8:121472376-121474017:-,-0.3443321,0.0030588371,0.3051057
chr9:118455515-118465694:+,-0.3429474,0.0031872924,0.3151858
chr10:39680947-39690086:+,-0.3415289,0.0033238563,0.3277512
chr9:75202443-75203442:+,-0.3393938,0.0035392431,0.3460237


In [7]:
run_analysis(
    "dea/esr1",
    dds = dds_esr1,
    name = "esr1"
)

Saving 6.67 x 6.67 in image
preparing geneSet collections...

GSEA analysis...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (65.45% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are duplicate gene names, fgsea may produce unexpected results.”
no term enriched under specific pvalueCutoff...



[1] "No significant GO terms"


class: DESeqDataSet 
dim: 34512 72 
metadata(1): version
assays(4): counts mu H cooks
rownames(34512): chr1:3451731-3451944:+ chr1:3638591-3638734:+ ...
  chrX_GL456233v2_random:409324-409604:+
  chrX_GL456233v2_random:409324-422124:+
rowData names(42): baseMean baseVar ... deviance maxCooks
colnames(72): aging_12m_ESR1_no_1 aging_12m_ESR1_no_2 ...
  antiHormonal_18m_ESR1_no_2 antiHormonal_18m_ESR1_no_3
colData names(8): condition age ... esr1 sizeFactor

# Drug effects

## Tamoxifen

In [8]:
dds_no_esr1 <- run_analysis(
    "dea/tamoxifen",
    phenotype = phenotype,
    design = ~ age + transgene + induction + drug,
    contrast=c("drug", "tamoxifen", "no")
)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Saving 6.67 x 6.67 in image
preparing geneSet collections...

GSEA analysis...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (3.12% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are duplicate gene names, fgsea may produce unexpected results.”
leading edge analysis...

done...

Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Warning message in cnetplot.enrichResult(x, ...):
“Use 'color.params = list(foldChange = your_value)' instead of 'foldChange'.
 The foldChange parameter will be removed in the next version.”
Scale for size is already present.
Adding another scale for size, whic

## Letrozole

In [9]:
run_analysis(
    "dea/letrozole",
    dds = dds_no_esr1,
    contrast=c("drug", "letrozole", "no")
)

Saving 6.67 x 6.67 in image
preparing geneSet collections...

GSEA analysis...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are duplicate gene names, fgsea may produce unexpected results.”
no term enriched under specific pvalueCutoff...



[1] "No significant GO terms"


class: DESeqDataSet 
dim: 34512 72 
metadata(1): version
assays(4): counts mu H cooks
rownames(34512): chr1:3451731-3451944:+ chr1:3638591-3638734:+ ...
  chrX_GL456233v2_random:409324-409604:+
  chrX_GL456233v2_random:409324-422124:+
rowData names(38): baseMean baseVar ... deviance maxCooks
colnames(72): aging_12m_ESR1_no_1 aging_12m_ESR1_no_2 ...
  antiHormonal_18m_ESR1_no_2 antiHormonal_18m_ESR1_no_3
colData names(8): condition age ... esr1 sizeFactor